# Scikit-Learn-Tagger (SKLTagger)

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import re

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_validate, cross_val_predict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Da unser Tagger mittlerweile recht komplex geworden ist, modularisieren wir ihn und implementieren die wesentlichen Bestandteile in einem lokalen Paket `skltagger`. Dadurch bleibt das Notebook für unsere Experimente übersichtlich und neue Merkmale können leicht eingebaut werden. Außerdem können wir so später auch leicht eine Kommandozeilenversion oder eine Python-Schnittstelle entwickeln.

Zumindest in der Entwicklungsphase werden wir oft neue Versionen z.B. der Merkmalsextraktion importieren wollen, ohne das Notebook komplett neu starten zu müssen. Das ist mit der `%autoreload`-Direktive möglich, die wir hier nur verwenden, um unsere eigenen Module neu zu laden. Ein Re-Import von Pandas, NumPy und SciPy in jeder Programmzelle könnte sonst die Ausführung drastisch verlangsamen.

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport skltagger.vectorizer
%aimport skltagger.classifier
%aimport skltagger.utils

Bei aktuellen Versionen von IPython/Jupyter sollte `%autoreload` auch mit `from ... import` funktionieren. Im Zweifelsfall ist es aber sicherer, alle Klassen und Hilfsfunktionen mit vollständig qualifizierten Pfaden aufzurufen (z.B. `skltagger.vectorizer.TaggerFeatures()`).

In [ ]:
from skltagger.vectorizer import TaggerFeatures
from skltagger.classifier import PseudoMarkovClassifier
from skltagger.utils import sentences2dataframe, load_model

## Trainings- und Testdaten

Laden und Vorverarbeitung der Trainings- (`_train`) und Testdaten (`_dev`) erfolgt wie gewohnt.

In [ ]:
header = ("sent", "tok", "word", "pos", "lemma", "morph")
train = pd.read_csv("data/tiger2_train.tsv.gz", sep="\t", names=header, quoting=3, keep_default_na=False)
test = pd.read_csv("data/tiger2_dev.tsv.gz",sep="\t",names=header, quoting=3, keep_default_na=False)

In [ ]:
train = train.iloc[:, 0:4]
test = test.iloc[:, 0:4]

In [ ]:
train.word = train.word.astype('string')
train.pos  = train.pos.astype('string')
test.word  = test.word.astype('string')
test.pos   = test.pos.astype('string')

## Verwendung der SKLTagger-Bibliothek

Wir haben die Merkmalsextraktion als Klasse mit Scikit-Learn-API implementiert und können sie dadurch in der gewohnten Weise verwenden. Größter Vorteil ist, dass unsere Klasse problemlos in Pipelines integriert werden kann (z.B. für Grid Search).

Training und Evaluation eines Taggers ist jetzt sehr übersichtlich.

In [ ]:
vectorizer = TaggerFeatures(shape_features=True)
X_train = vectorizer.fit_transform(train)
X_test = vectorizer.transform(test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
%%time
clf = LinearSVC()
clf.fit(X_train, train.pos)
clf.score(X_train, train.pos)

In [ ]:
predicted = clf.predict(X_test)
print(accuracy_score(test.pos, predicted))
print(classification_report(test.pos, predicted, digits=3, zero_division=0))

Wir können die Merkmalsextraktion auch in eine **Pipeline** integrieren, die dann bereits einen vollständigen POS-Tagger bildet und als Parameterdatei abgespeichert werden kann. Die Evaluation des Taggers auf den Trainings- und Testdaten ist jetzt natürlich deutlich langsamer, da die Merkmalsextraktion jedes Mal erneut durchgeführt werden muss.

Wir verwenden dazu Stochastic Gradient Descent als Lernverfahren, das dank Parallelisierung wesentlich schneller trainiert werden kann. Es liefert aber bisweilen etwas schlechtere Ergebnisse als die SVM und ist weniger robust ohne Optimierung der Metaparameter. Wenn wir später Kreuzvalidierung und Grid Search verwenden, relativiert sich der Geschwindigkeitsvorteil ohnehin, so dass die SVM unser bevorzugtes Lernverfahren bleiben dürfte.

In [ ]:
tagger = Pipeline([
    ('vect', TaggerFeatures()),
    ('clf', SGDClassifier(loss='log_loss', alpha=1e-6, max_iter=5000, n_jobs=-1)),
])
%time tagger.fit(train, train.pos)
%time tagger.score(train, train.pos)

**Aufgabe:** Wie schnell ist der Tagger auf Ihrem Rechner? D.h. wie viele Token / s werden verarbeitet?

In [ ]:
%time tagger.score(test, test.pos)

Um den Tagger nicht nur auf das Testkorpus, sondern auch beliebige **andere Texte** anwenden zu können, müssen wir diese in unser Pandas-Format überführen. Wir haben dazu eine Hilfsfunktion definiert, die eine Liste von tokenisierten Sätzen transformiert.

In [ ]:
text = [
    "Hunde , die schlafen , bellen nicht !".split(),
    "Peter streichelt die Hunde .".split()
]
print(text)
text = sentences2dataframe(text)

In [ ]:
text['pos'] = tagger.predict(text)
text

## Kreuzvalidierung und Tuning

Pipelines sind eine Voraussetzung für die saubere Kreuzvalidierung des Taggers, da auch die Merkmalsextraktion jeweils nur auf der als Trainingsdaten ausgewählten Teilmenge trainiert werden darf. Diese Kreuzvalidierung wird wiederum für die Optimierung der Metaparameter gebraucht (bzw. ersetzt dort ein separates Validation Set).

Da jede Fold aus vollständigen Sätzen bestehen soll, können wir nicht die Standardaufteilung verwenden sondern benötigen ein `GroupKFold`-Objekt. Dabei stellt jeder Satz eine eigene Gruppe (repräsentiert durch die Satznummer) dar. Die Warnmeldungen bei `cross_validate` lassen sich leider nicht vermeiden.

In [ ]:
group_cv = GroupKFold(n_splits=3)
cross_validate(tagger, train, train.pos, cv=group_cv, groups=train.sent, 
               scoring=('accuracy', 'precision_macro', 'recall_macro', 'f1_macro'), 
               return_train_score=True)

Mit Hilfe einer Pipeline können wir auch eine Grid search durchführen, mit der sowohl Merkmalsextraktion als auch das maschinelle Lernverfahren zugleich optimiert werden. Da die Anzahl der zu testenden Metaparameterkombinationen aber schnell explodiert, führt man diese Optimierung in der Praxis oft in mehreren Schritten durch, bei denen einige Parameter bereits festgelegt werden, bzw. Wertebereiche (etwa für $\alpha$) in einem zweiten Durchlauf verfeinert werden.

In [ ]:
%%time
grid_pipe = Pipeline([
    ('vect', TaggerFeatures()),
    ('sgd', SGDClassifier(loss='log_loss', max_iter=5000, n_jobs=-1)),
])
param_grid = {
    'vect__left_context': [1, 2],
    'vect__right_context': [0, 1],
    'vect__shape_features': [True, False],
    'sgd__alpha': [1e-5, 1e-6, 1e-7],
}

gs = GridSearchCV(grid_pipe, param_grid, scoring='accuracy', refit='accuracy',
                  cv=group_cv, n_jobs=1, verbose=3) # SGD ist bereits parallelisiert
gs.fit(train, train.pos, groups=train.sent);

Wir können nun die optimalen Parameter auslesen und das beste Modell (das bereits auf dem kompletten Datensatz trainiert wurde) auf den Testdaten evaluieren.

In [ ]:
print(gs.best_params_)
print(gs.best_estimator_.score(test, test.pos))

Eine detailliertere Auswertung der Grid Search, um den Einfluss verschiedener Parameter bzw. Parameterkombinationen zu untersuchen, kann am besten mit Pandas durchgeführt werden. So sehen wir z.B., dass Kontextinformation sowohl von der linken als auch der rechten Seite unbedingt notwendig ist, dass aber mehr als ein Token Kontext keine weitere Verbesserung bringt.

In [ ]:
gs_info = pd.DataFrame(gs.cv_results_)
gs_info.drop(['params', 'std_fit_time', 'std_score_time'] +
             [col for col in gs_info.columns if col.startswith('split')], axis=1)

## Sequenzmodellierung

Unser Tagger verwendet zwar Kontextinformation, trifft aber dennoch für jedes Token eine separate Entscheidung und berücksichtigt nicht, ob die zugewiesenen Tags insgesamt eine plausibles Satzmuster ergeben (im Gegensatz z.B. zum HMM-Tagger). Auch kann er, wie wir gerade gesehen haben, keinen größeren Kontext ausnutzen als die beiden unmittelbar angrenzenden Token.

Wir haben zuletzt ein Lernverfahren verwendet (SGD als logistische Regression), das nicht nur eine Tagging-Entscheidung trifft, sondern auch eine Wahrscheinlichkeitsverteilung über alle möglichen Tags lieferen kann. Das ermöglicht im Prinzip eine Kombination mit einem N-Gramm-Modell auf Ebene der Tags, so dass mit dem Viterbi-Algorithmus die plausibelste Abfolge von Tags ausgewählt werden kann – wobei unser Lernverfahren $P(t_i | w_i, w_{i-1}, w_{i+1})$ beisteuert und das N-Gramm-Modell $P(t_i | t_{i-1}, t_{i-2})$. Dabei handelt es sich um eine Annäherung an ein HMM-Modell (wegen der kontextabhängigen lexikalischen Wahrscheinlichkeiten werden dessen Unabhängigkeitsannahmen allerdings verletzt). Statistisch valide wäre eine Kombination mit einem _Conditional Random Field_ (CRF) als Sequenzmodell. 

Scikit-Learn bietet leider weder HMM- noch CRF-Modelle an, da [diese nicht zur Scikit-Learn-API passen](https://scikit-learn.org/stable/faq.html#adding-graphical-models) würden. Wir behelfen uns daher mit einer Approximation, die analog zum HMM die Wahrscheinlichkeiten von Tag-N-Grammen lernen kann, aber immer noch eine unabhängige Entscheidung für jedes Token trifft. Wir implementieren diesen Ansatz als eine zweite Lernebene, die als Merkmale Wahrscheinlichkeitsverteilungen $P(t_i | w_i, w_{i-1}, w_{i+1})$ über POS-Tags verwendet, die von unserem bisherigen Lernmodel erstellt werden. Die Verteilung für das aktuell zu taggende Token wird dann mit den Verteilungen der umliegenden Token kombiniert, so dass das Lernverfahren einen Tag wählen kann, der sowohl zur lexikalischen Information als auch zu wahrscheinlichen POS-Tags der umliegenden Token passt.

Als erstes trainieren wir noch einmal einen geeigneten Classifier, der uns die benötigten Wahrscheinlichkeitsverteilungen über alle möglichen Tags liefern kann. Er stellt die erste Stufe unseres Klassifikationsverfahrens dar. Um sinnvolle Wahrscheinlichkeitswerte für die Trainingsdaten zu erhalten, müssen wir die `predict_proba()`-Methode im Rahmen einer Kreuzvalidierung anwenden.

**Q:** Warum ist die Kreuzvalidierung hier unbedingt erforderlich? Was würde die zweite Stufe lernen, wenn wir `predict_proba()` ohne Kreuzvalidierung anwenden?

An dieser Stelle taucht ein praktisches Problem auf: manche Tags sind so selten, dass sie bei der Kreuzvalidierung möglicherweise im jeweiligen Trainingsdatensatz nicht vorkommen. Aus diesem Grund hätten wir oben auch besser `StratifiedGroupKFold` statt `GroupKFold` verwenden sollen, was hier allerdings nicht weiterhilft (da das Tag `VAIMP` in unseren gesamten Trainingsdaten nur ein einziges Mal vorkommt). Bisher haben wir das Problem einfach ignoriert und die resultierenden Warnungen in Kauf genommen.

**Q:** Warum würde die zweite Stufe unseres Klassifikationsverfahren überhaupt nicht mehr funktionieren, wenn wir das Problem weiter ignorieren?

Praktikabelste Lösung ist, sehr seltene Tags in den Trainingsdaten zu ersetzen: `VAIMP` duch `VAFIN` und `NNE` (ein Tippfehler) durch `NE`. Wir sollten trotzdem eine stratifizierte Kreuzvalidierung machen, da sonst z.B. `VMPP` (mit $f=4$) in den jeweiligen Trainingsdaten fehlen könnte.

In [ ]:
train_pos_fixed = train.pos.mask(train.pos == 'VAIMP', 'VAFIN').mask(train.pos == 'NNE', 'NE')

Der Einfachheit halber führen wir die Merkmalsextraktion direkt auf den gesamten Trainingsdaten aus und wenden die Kreuzvalidierung nur auf den Classifier an. Dadurch können wir auch die voreingestellte Kreuzvalidierung verwenden, die automatisch stratifiziert.

In [ ]:
vect = TaggerFeatures()
X = vect.fit_transform(train)

In [ ]:
clf1 = SGDClassifier(loss='log_loss', alpha=1e-6, max_iter=5000, n_jobs=-1)
X_prob = cross_val_predict(clf1, X, train_pos_fixed, cv=2, method='predict_proba')
X_prob.shape

In [ ]:
clf1.fit(X, train_pos_fixed); # für Anwendung auf Testdaten

Nun müssen wir die 52-dimensionalen Vektoren mit den Wahrscheinlichkeitsverteilungen über POS-Tags um die entsprechenden Wahrscheinlichkeitsverteilungen für die vorhergehenden und folgenden Token ergänzen. Dazu verschieben wir alle Spalten der Merkmalsmatrix um entsprechend viele Positionen. Theoretisch sollte das ebenfalls satzweise mit Padding geschehen. Um nicht auf die Originaldaten zurückgreifen zu müssen, ignorieren wir die Satzgrenzen und **rotieren** die Spalten (d.h. die untersten Elemente werden oben wieder angefügt und umgekehrt). Diese Operation ist auch direkt in NumPy implementiert, so dass wir nicht auf Pandas oder SciPy ausweichen müssen. Wir definieren dazu eine Hilfsfunktion, da wir sowohl Trainings- als auch Testdaten entsprechend bearbeiten müssen.

In [ ]:
def add_contexts(X):
    return np.hstack([
        X,                      # aktuelles Token
        np.roll(X, 1, axis=0),  # erstes Token links
        np.roll(X, 2, axis=0),  # zweites Token links
        np.roll(X, -1, axis=0), # erstes Token rechts
        np.roll(X, -2, axis=0), # zweites Token rechts
    ])

X_prob = add_contexts(X_prob)
X_prob.shape

Da die zweite Stufe analog zum HMM die Wahrscheinlichkeiten verschiedener POS-N-Gramme lernen soll benötigen wir einen Algorithmus, der Merkmalskombinationen berücksichtigen kann. Beispielsweise wäre eine SVM mit polynomialem Kern grundsätzlich sehr gut geeignet, ist allerdings für unsere großen Trainings- und Testdatensätze viel zu ineffizient. 

Hier verwenden wir ein Ensemble von Entscheidungsbäumen, das als **Random Forest** bekannt ist. (Freiwillige **Zusatzaufgabe:** Ein alternativer Ansatz besteht darin, einen effizienten linearen Classifier – z.B. SGD – mit einer [Nystroem-Kernelapproximation](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html) zu kombinieren. Experimentieren Sie mit diesem Ansatz und unterschiedlichen Parametern der Nystroem-Approximation.)

In [ ]:
%%time
clf2 = RandomForestClassifier(n_jobs = -1)
clf2.fit(X_prob, train.pos)
clf2.score(X_prob, train.pos)

Zur Evaluation des Random-Forest-Classifiers müssen wir für die Testdaten ebenfalls Wahrscheinlichkeitsverteilungen für aktuelles Token und Kontext erstellen.

In [ ]:
X_prob_test = add_contexts(clf1.predict_proba(vect.transform(test)))
X_prob_test.shape

In [ ]:
clf2.score(X_prob_test, test.pos)

Unser einfacher Ansatz zur Sequenzmodellierung konnte tatsächlich eine merkliche, wenn auch nicht sonderlich große Verbesserung erzielen und die Tagging-Genauigkeit auf knapp 96,8% steigern. Mit optimierten Metaparametern des `RandomForestClassifier` oder zusätzlichem Kontext wäre vielleicht sogar eine weitere Steigerung möglich.

**Aufgabe:** Eine Alternative wäre, die Tag-Wahrscheinlichkeiten der Kontext-Token mit der ursprünglichen Merkmalsmatrix zu kombinieren und den zweiten Classifier damit zu trainieren. Da nur lineare Classifier effizient genug mit dieser großen Merkmalsmatrix umgehen, können damit keine N-Gramme von Tags simuliert werden; trotzdem liefern einzelne Tags im Kontext vielleicht nützliche Hinweise zur Desambiguierung. Implementieren Sie diesen Ansatz, wobei der erste Classifier auch einfach die wahrscheinlichsten Tags zuweisen könnte statt die vollen Wahrscheinlichkeitsverteilungen zu berechnen.


## Sequenzmodellierung als Pipeline-Komponente

Für weitere Experimente und die praktische Anwendung des Taggers wollen wir unsere zweistufige Approximation der Sequenzmodellierung natürlich als Classifier-Modul in eine Scikit-Learn-Pipeline integrieren. Da eine Pipeline nur einen einzigen Classifier enthalten darf, muss der zweistufige Prozess in ein Modul kombiniert werden.

Eine Variante des oben erwähnten alternativen Ansatzes ist in der Klasse `PseudoMarkovClassifier` im Modul `skltagger.classifier` implementiert. Wir verwenden diese Klasse hier, um unseren finalen Tagger zu trainieren und auf den Testdaten zu evaluieren.

`PseudoMarkovClassifier` verwendet in der ersten Stufe logistische Regression (mit SGD trainiert), um Tag-Wahrscheinlichkeitsverteilungen für die Kontext-Token zu bestimmen. In der zweiten Stufe kann dann ein beliebiger Classifier verwendet werden, der bei der Instanziierung mit übergeben wird. Die Metaparameter dieses Classifiers müssen vorher bereits festgelegt werden und können über das `PseudoMarkovClassifier`-Objekt nicht mehr verändert werden.

Als Beispiel erstellen wir hier eine Pipeline, die die üblichen SKLTagger-Merkmale erstellt und dann unseren zweistufigen Classifier mit einer SVM in der zweiten Stufe anwendet. SVMs haben den Vorteil, dass oft auf ein umfangreiches Tuning der Metaparameter verzichtet werden kann. Wichtige Parameter von `PseudoMarkovClassifier` sind `n_jobs` für Parallelisierung der ersten Stufe sowie die Anzahl der berücksichtigten Kontext-Token. Auch bei relativ großem Kontext entsteht kein hoher Zusatzaufwand, so dass wir hier je 5 Token links und rechts hinzunehmen. (NB: `n_jobs` bezieht sich nur auf die logistische Regression der ersten Stufe und muss ggf. beim Classifier der zweiten Stufe separat angegeben werden. Die vollen Klassenpfade sind notwendig, damit die Serialisierung des trainierten Modells auch bei aktiviertem `autoreload` zuverlässig funktioniert.)

In [ ]:
svm = LinearSVC(loss='hinge', C=0.1, max_iter=2000)
tagger = Pipeline([
    ('vect', skltagger.vectorizer.TaggerFeatures()),
    ('pmclf', skltagger.classifier.PseudoMarkovClassifier(svm, left_context=5, right_context=5, n_jobs=-1)),
])

In [ ]:
%%time
tagger.fit(train, train.pos)
tagger.score(train, train.pos)

**Aufgabe:** Falls Ihnen das Training der SVM zu lange dauert, können Sie stattdessen einen `SGDClassifier` mit `loss='hinge'` einsetzen. Denken Sie daran, hier im Konstruktor auch `n_jobs=-1` anzugeben, damit beide Stufen parallelisiert werden!

Die Evaluation auf dem Testkorpus zeigt mit fast **97,3% Genauigkeit** sehr gute Ergebnisse. Der neue Tagger ist zwar deutlich langsamer als das einstufige Verfahren, kann aber immer noch ca. 20k Token / s verarbeiten (_your mileage may vary_).

In [ ]:
%%time
tagger.score(test, test.pos)

## Kommandozeilen-Interface

Um unseren neuen Tagger praktisch einsetzen zu können, müssen wir das trainierte Modell speichern und später wieder laden können. Wie von der Scikit-Learn-Dokumentation empfohlen setzen wir dafür Funktionen aus dem `joblib`-Paket ein. 

In [ ]:
from joblib import dump, load
dump(tagger, 'german-tagger.pkl') 

Beim Laden eines Modells sollte überprüft werden, dass es sich tatsächlich um einen SKLTagger handelt. Dazu stellt das `skltagger.utils`-Modul die Funktion `load_model()` bereit.

In [ ]:
load_model('german-tagger.pkl')

Schließlich wollen wir ein Kommandozeilen-Interface (_command-line interface_, **CLI**) bereitstellen, mit dem ein trainiertes Tagger-Modell auf Textdateien angewendet werden kann. Es ist üblich (und empfehlenswert), die benötigte Funktionalität in einem separaten Modul (`skltagger.cli`) zu implementieren, so dass sie auch von anderen Python-Programmen genutzt werden kann. Wir verzichten dabei hier auf die eigentlich selbstverständliche ausführliche Dokumentation des Kommandozeilen-Taggers.

Der Tagger soll entweder vom Nutzer eingegebene Sätze oder eine ganze Textdatei verarbeiten und muss daher einen geeigneten Tokenizer (hier: SoMaJo) integrieren. Das Tokenizer-Paket muss nur installiert sein, wenn `skltagger.cli` geladen wird. Kern des CLI-Moduls ist eine Funktion, die eine Zeichenkette in Sätze zerlegt, tokenisiert und mit der ebenfalls übergebenen Pipeline taggt.

In [ ]:
import skltagger.cli
skltagger.cli.tag_text(tagger, 'Hunde, die schlafen, bellen nicht! Peter streichelt die Hunde.')

Wird das Modul als Skript aufgerufen, dann fungiert es als Kommandozeilen-Programm. Mit der Option `-h` können alle Optionen angezeigt werden:

    python -m skltagger.cli -h

Als Parameter muss immer die in eine Datei gespeicherte SKLTagger-Pipeline angegeben werden. Ohne weitere Parameter können interaktiv Sätze eingegeben und getaggt werden.

    python -m skltagger.cli german-tagger.pkl

Es können auch ein oder mehrere Textdateien mit der Option `-i` übergeben werden. Alle Texte werden konkateniert und im _vertical-text_-Format auf STDOUT ausgegeben. Ein typischer Aufruf würde also so aussehen:

    python -m skltagger.cli german-tagger.pkl -i text1.txt -i text2.txt -i text3.txt > text.vrt
